In [1]:
import sys
import pandas as pd
sys.path.append(r'C:\Users\alexa\OneDrive\Documentos\R\Banco de Dados\Sidra-PI\dags')

from _managers import (GoogleSheetManager, GoogleDriveManager)
from _estrutures import (InformationBaseManager)
from sidrapi import (Sidra_Data)
import json

In [2]:
INFORMATION_PATH = r"C:\Users\alexa\OneDrive\Documentos\R\Banco de Dados\Sidra-PI\data\__data__info.json"
DATA_PATH = r"C:\Users\alexa\OneDrive\Documentos\R\Banco de Dados\Sidra-PI\data\processed"

with open(INFORMATION_PATH, 'r') as file:
    info_data = json.load(file)

In [3]:
CREDENTIALS_PATH = info_data["Local Credentials"]
MAIN_FOLDER = info_data["MainFolder"]["Name"]
KNOWLEDGE_BASE = info_data["Knowledge base"]

# Classes
sheet_manager = GoogleSheetManager(CREDENTIALS_PATH)
drive_manager = GoogleDriveManager(CREDENTIALS_PATH, INFORMATION_PATH)
information_manager = InformationBaseManager(KNOWLEDGE_BASE)
sidra_manager = Sidra_Data()

In [4]:
df_tabelas_concatenados = pd.read_excel(r"C:\Users\alexa\OneDrive\Documentos\R\Banco de Dados\Sidra-PI\data\processed\dados_sidra.xlsx", sheet_name="Tabelas")
df_variaveis_concatenados = pd.read_excel(r"C:\Users\alexa\OneDrive\Documentos\R\Banco de Dados\Sidra-PI\data\processed\dados_sidra.xlsx", sheet_name="Variáveis")

In [5]:
for idx, row in df_tabelas_concatenados.iterrows():
    num_tb = row["Número da Tabela"]
    tb_var = df_variaveis_concatenados[df_variaveis_concatenados["Número da Tabela"] == num_tb]

    pages_data = []
    pages_names = []

    for i, row_var in tb_var.iterrows():
        df = sidra_manager.fetch_sidra_data(t=num_tb, 
                                            v=row_var["Número da Variável"],
                                            c=row["Classificador da Tabela"])
        pages_data.append(df)
        nome_aba = row_var["Descrição"].split("-")[0]
        # Removendo caracteres inválidos e limitando o comprimento
        nome_aba = nome_aba.translate({ord(c): None for c in '\\/*?:"[]'})
        nome_aba = nome_aba[:31]  # Limitando a 31 caracteres
        pages_names.append(nome_aba)

    # Salvando os DataFrames como abas diferentes em um arquivo Excel
    with pd.ExcelWriter(f'{DATA_PATH}\\{row["Nome da Tabela"][:20]}.xlsx', engine='openpyxl') as writer:
        for df, nome_aba in zip(pages_data, pages_names):
            df.to_excel(writer, sheet_name=nome_aba, index=False)
